In [1]:
import json
import random

import nltk
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#nltk.download('punkt_tab')

In [ ]:
"""# load the dataset
with open('data/healthcare.json','r',encoding='utf-8') as f:
    data = json.load(f)

intents = {'intents':[]}
#
for convo in data:
    tag = convo.get("agent_selected_tool", "general").replace(" ","_").lower()


    pattern = convo.get("user_1", "")
    response = convo.get("agent_initial_response", "")


    if pattern and response:
        intents['intents'].append({
            "tag":tag,
            "patterns":[pattern],
            "responses":[response]
        })

# Save the new dataset
with open("data/healthcare_intents.json", "w", encoding="utf-8") as f:
    json.dump(intents, f, indent=4)


print('Dataset intentions create with sucess!')"""
    








Dataset intentions create with sucess!


In [ ]:
class ChatboModel(nn.Module):
    
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size,)
        self.fc2 = nn.Linear()
        self.fc3 = nn.Linear(,output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = self.relu(self,x)
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x


In [ ]:
class ChabotAssistant:
    def __init__(self,intents_path, function_mappings = None):
        self.model = None
        self.intents_path = intents_path

        self.documents = []





        